In [1]:
from CybORG import CybORG
from CybORG.Simulator.Scenarios import EnterpriseScenarioGenerator
from CybORG.Agents.Wrappers import EnterpriseMAE
from CybORG.Agents import SleepAgent, EnterpriseGreenAgent, FiniteStateRedAgent
from Wrappers import EnterpriseMAEMaskWrapper, MaskWrapper
from CustomRLLib import TorchActionMaskModel, CustomModel

In [7]:
from ray.tune import register_env
from ray.rllib.algorithms.ppo import PPOConfig, PPO
from ray.rllib.algorithms.dqn import DQNConfig
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.models import ModelCatalog
import time

In [3]:
def env_creator_CC4(env_config: dict):
    sg = EnterpriseScenarioGenerator(
        blue_agent_class=SleepAgent,
        green_agent_class=EnterpriseGreenAgent,
        red_agent_class=FiniteStateRedAgent,
        steps=500
    )
    cyborg = CybORG(scenario_generator=sg)
    # env = EnterpriseMAE(env=cyborg)
    # env = EnterpriseMAEMaskWrapper(env=cyborg)
    env = MaskWrapper(env=cyborg)

    return env

In [4]:
register_env(name='CC4', env_creator=lambda config: env_creator_CC4(config))
env = env_creator_CC4({})

In [5]:
NUM_AGENTS = 5
POLICY_MAP = {f"blue_agent_{i}": f"Agent{i}" for i in range(NUM_AGENTS)}

def policy_mapper(agent_id, episode, worker, **kwargs):
# def policy_mapper(agent_id, episode, **kwargs):
    return POLICY_MAP[agent_id]

In [6]:
ModelCatalog.register_custom_model("custom_pytorch_model", CustomModel)
ModelCatalog.register_custom_model("torch_action_mask_model", TorchActionMaskModel)
# ModelCatalog.register_custom_model("action_mask_model", ActionMaskModel)


# ray.init()
# algo = ppo.PPO(env="CartPole-v1", config={
#     "model": {
#         "custom_model": "my_tf_model",
#         # Extra kwargs to be passed to your model's c'tor.
#         "custom_model_config": {},
#     },
# })

In [7]:
algo_config = (
    PPOConfig()
    .environment(env="CC4")
    .framework("torch")
    .debugging(logger_config={'logdir':'./logs/Masked_PPO_Example', 'type': 'ray.tune.logger.TBXLogger'})
    .multi_agent(policies={
        ray_agent: PolicySpec(
            policy_class=None,
            observation_space=env.observation_space(cyborg_agent),
            action_space=env.action_space(cyborg_agent),
            config={'gamma': 0.85}
        ) for cyborg_agent, ray_agent in POLICY_MAP.items()
    },
    policy_mapping_fn=policy_mapper
    )
    .training(
        model={'custom_model': "torch_action_mask_model"}
    )
)

In [8]:
print(algo_config.policies['Agent0'].policy_class)

None


In [9]:
import time
print(time.strftime("%Y-%m-%d_%H:%M:%S"))

2024-04-16_23:54:54


## Masked

In [15]:
algo = algo_config.build()
num_iters = 50
for i in range(num_iters):
    train_info=algo.train()

algo.save(f'./Submissions/mask_results_{num_iters}_{time.strftime("%Y-%m-%d_%H:%M:%S")}/staging/')

/Applications/anaconda3/envs/cage-4/lib/python3.10/site-packages/ray/rllib/utils/from_config.py:197: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `TBXLogger interface is deprecated in favor of the `ray.tune.tensorboardx.TBXLoggerCallback` interface and will be removed in Ray 2.7.
  object_ = constructor(*ctor_args, **ctor_kwargs)
2024-04-17 01:29:19,648	WARNING util.py:62 -- Install gputil for GPU system monitoring.


!!!! space_to_dict(self.observation_space):  {'space': {'space': 'box', 'low': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RpGZuY6mjoKtQpkAy4Ghob9o3gUj+JRTAgDAJnVaEo=', 'high': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RpGZuY6mjoKtQpkAy4Ghgb7UTyKR/EoJoQBlgviuw==', 'shape': (267,), 'dtype': '<f4'}, 'original_space': {'space': {'space': 'dict', 'spaces': {'action_mask': {'space': 'multi-binary', 'n': 82}, 'observations': {'space': 'multi-discrete', 'nvec': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2mhrqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RqWRjqaOgq1CuQDLmYGCGAapUfpYUgDAGpCHDI=', 'dtype': '<i8'}}}}}
!!!! space_to_dict(self.observation_space):  {'space': {'space': 'box', 'low': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RpGZuY6mjoKtQpkAy4Ghob9o3gUj+JRTAgDAJnVaEo=', 'high': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RpGZuY6mjoKtQpkAy4Ghgb7UTyKR/EoJoQBlgviuw=

TrainingResult(checkpoint=Checkpoint(filesystem=local, path=./Submissions/mask_results_50_2024-04-17_02:24:21/staging/), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'Agent3': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 8.562258143598834, 'cur_kl_coeff': 0.6750000000000002, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 8.699180987974009, 'policy_loss': -0.061590733847212205, 'vf_loss': 8.7496846879522, 'vf_explained_var': 0.5017867753903071, 'kl': 0.016425228798326495, 'entropy': 3.0372680140038333, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.0, 'num_grad_updates_lifetime': 47520.5, 'diff_num_grad_updates_vs_sampler_policy': 479.5}, 'Agent0': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 7.050541699305176, 'cur_kl_coeff': 1.0124999999999997, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 8.759998994568983, 'policy_loss': -0.04837757342271895, 'vf_loss': 8.797991787890593, 'vf_explain

In [11]:
policy = algo.get_policy(policy_id='Agent0')
# policy.compute_actions()
print(env.observation_space('blue_agent_0'))
print(len(env.observation_space('blue_agent_0').sample()))
print(env.action_space('blue_agent_0'))
print(policy.observation_space)

Dict('action_mask': MultiBinary(82), 'observations': MultiDiscrete([3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]))
2
Discrete(82)
Box(-1.0, 1.0, (267,), float32)


In [12]:
print(env.observation_space('blue_agent_0').sample())

OrderedDict([('action_mask', array([1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0], dtype=int8)), ('observations', array([2, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0]))])


In [13]:
print(env.observation_space('blue_agent_4'))

Dict('action_mask': MultiBinary(242), 'observations': MultiDiscrete([3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]))


In [14]:
from ray.rllib.policy.policy import PolicySpec, Policy
my_restored_policy = Policy.from_checkpoint("./Submissions/mask_results_2024-04-16_02:01:29/staging/policies/Agent0")

## Regular PPO

In [8]:
def ppo_env_creator_CC4(env_config: dict):
    sg = EnterpriseScenarioGenerator(
        blue_agent_class=SleepAgent,
        green_agent_class=EnterpriseGreenAgent,
        red_agent_class=FiniteStateRedAgent,
        steps=500
    )
    cyborg = CybORG(scenario_generator=sg)
    env = EnterpriseMAE(env=cyborg)
    # env = EnterpriseMAEMaskWrapper(env=cyborg)
    # env = MaskWrapper(env=cyborg)

    return env

In [9]:
register_env(name='CC4_PPO', env_creator=lambda config: ppo_env_creator_CC4(config))
env = ppo_env_creator_CC4({})

In [10]:
NUM_AGENTS = 5
POLICY_MAP = {f"blue_agent_{i}": f"Agent{i}" for i in range(NUM_AGENTS)}

def policy_mapper(agent_id, episode, worker, **kwargs):
# def policy_mapper(agent_id, episode, **kwargs):
    return POLICY_MAP[agent_id]

In [15]:
timestamp = time.strftime("%Y-%m-%d_%H:%M:%S")
ppo_algo_config = (
    PPOConfig()
    .environment(env="CC4_PPO")
    .framework("torch")
    .debugging(logger_config={'logdir':f'./tb_logs/PPO_Example_{timestamp}', 'type': 'ray.tune.logger.TBXLogger'})
    .multi_agent(policies={
        ray_agent: PolicySpec(
            policy_class=None,
            observation_space=env.observation_space(cyborg_agent),
            action_space=env.action_space(cyborg_agent),
            config={'gamma': 0.85}
        ) for cyborg_agent, ray_agent in POLICY_MAP.items()
    },
    policy_mapping_fn=policy_mapper
    )
)

In [16]:
ppo_algo = ppo_algo_config.build()
num_iters = 2
for i in range(num_iters):
    train_info=ppo_algo.train()

ppo_algo.save(f'./Submissions/results_{num_iters}_{timestamp}/staging/')

2024-04-17 09:40:57,934	WARNING util.py:62 -- Install gputil for GPU system monitoring.


!!!! space_to_dict(self.observation_space):  {'space': {'space': 'box', 'low': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RomRoY6mjoKtQpkAy4Ghob9o3gUj+JRPIpH8WDAAN8+KDc=', 'high': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RomRoY6mjoKtQpkAy4Ghgb7UTyKR/EoHsWjeDBgADBBVag=', 'shape': (421,), 'dtype': '<f4'}, 'original_space': {'space': {'space': 'multi-discrete', 'nvec': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2mhrqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RpGhgY6mjoKtQpkAy5mBghgGqVH6VF6lB6lR+kBpAFVox0m', 'dtype': '<i8'}}}
!!!! space_to_dict(self.observation_space):  {'space': {'space': 'box', 'low': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RqGFqY6mjoKtQpkAy4Ghob9o3gUD2cMAKaKAhs=', 'high': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RqGFqY6mjoKtQpkAy4Ghgb7UTyKhzMGAEo/pYw=', 'shape': (185,), 'dtype': '<f4'}, 'original_space': {'space': {'space': 'multi-discret

TrainingResult(checkpoint=Checkpoint(filesystem=local, path=./Submissions/results_2_2024-04-17_09:40:49/staging/), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'Agent4': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.9237068885316452, 'cur_kl_coeff': 0.29999999999999993, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 9.856803868214289, 'policy_loss': -0.0709566344313013, 'vf_loss': 9.921804336706797, 'vf_explained_var': -1.4175350467363993e-06, 'kl': 0.019853963697676478, 'entropy': 5.445857546230157, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.0, 'num_grad_updates_lifetime': 1440.5, 'diff_num_grad_updates_vs_sampler_policy': 479.5}, 'Agent1': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.9096082871779799, 'cur_kl_coeff': 0.29999999999999993, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 9.8861259440581, 'policy_loss': -0.040507729798264336, 'vf_loss': 9.921859556436539, 'vf_explain